In [1]:
import pandas as pd

In [2]:
data_users = "data/BX-Users_upr.csv"
data_books = "data/BX-Books_upr.csv"
data_ratings = "data/BX-Book-Ratings_upr.csv"

In [3]:
class Data:
    """
    Load database about users, books and book's ratings.
    Data are supposed in format:
    users: User-ID; Location; Age
    books: ISBN; Book-Title; Book-Author; Year-Of-Publication; Publisher; Image-URL-S; Image-URL-M-IMAGE-URL-L
    ratings: User-ID; ISBN; Book-Rating
    """
    def __init__(self, users, books, ratings):
        #self.users = pd.read_csv(users, sep=";", engine="python")
        self.books = pd.read_csv(books, sep=";", engine="python")
        self.ratings = pd.read_csv(ratings, sep=";", engine="python")
        
    def get_isbn(self, book_name, book_author = None):
        isbn = self.books[self.books["Book-Title"].str.contains(book_name)][["ISBN", "Book-Title", "Book-Author"]]
        if len(isbn) == 0:
            print("Book is not in database")
            return
        if book_author:
            isbn = isbn[isbn["Book-Author"].str.contains(book_author)]
        return [isbn["ISBN"].iloc[i] for i in range(len(isbn))]
        
    def get_users_who_like(self, isbn, rating = 7):
        users_who_like = self.ratings[self.ratings["ISBN"] == isbn]
        users_who_like = users_who_like[users_who_like["Book-Rating"] >= rating]
        return [users_who_like["User-ID"].iloc[i] for i in range(len(users_who_like))]
    
    def find_rated_books(self, users_who_like, my_book):
        rated_books = self.ratings[self.ratings["User-ID"] == users_who_like[0]]            
        for i in range(1, len(users_who_like)):
            rated_books2 = self.ratings[self.ratings["User-ID"] == users_who_like[i]]
            rated_books = pd.concat([rated_books, rated_books2])
        # TODO rozdelit
        for j in range(len(my_book)):    
            for i in range(len(rated_books[rated_books["ISBN"] == my_book[j]])):
                c = rated_books[rated_books["ISBN"] == my_book[j]].index[0]
                rated_books = rated_books.drop(index=c)
        return rated_books
    
    def get_books_rating(self, rated_books):
        books_rating = []
        while len(rated_books):
            book = rated_books[rated_books["ISBN"] == rated_books["ISBN"].iloc[0]]
            books_rating.append([book["ISBN"].iloc[0], sum(book["Book-Rating"]), len(book)])
            for i in range(len(book)):       
                rated_books = rated_books.drop(index=book.index[i])
        return books_rating
    
    def get_recommended_book(self, books_rating):
        max_rating = 0
        for i in range(len(books_rating)):
            if(books_rating[i][1] > max_rating):
                max_rating = books_rating[i][1]
                the_book = books_rating[i]
        return the_book    
    
    def get_book_by_isbn(self, isbn):
        return self.books[self.books["ISBN"] == isbn]

In [4]:
class Book:
    def __init__(self, isbn, title, author, year_public = None, publisher = None):
        self.isbn = isbn
        self.title = title
        self.author = author
        self.year_public = year_public
        self.publisher = publisher
        self.ratings = 0
        self.rating_users = 0

In [8]:
data = Data(data_users, data_books, data_ratings)
#favourite_book_name = "Lord of the Rings"
favourite_book_name = "Hobbit"
favourite_book_author = "Tolkien"

In [6]:
#isbn = data.get_isbn(favourite_book_name, favourite_book_author)
isbn = data.get_isbn(favourite_book_name)

In [7]:
my_book = Book(isbn[0], favourite_book_name, favourite_book_author)
users_who_like = data.get_users_who_like(isbn[0])
rated_books = data.find_rated_books(users_who_like, isbn)
books_rating = data.get_books_rating(rated_books)
recommended_book = data.get_recommended_book(books_rating)

NameError: name 'favourite_book_author' is not defined

In [ ]:
data.get_book_by_isbn(recommended_book[0])

In [ ]:
data.books[data.books.ISBN == "0374157065"].index